In [7]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [26]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO
import pprint

In [27]:
g = Graph()
g.parse("D:\\Downloads\\bioport_xml_batch_1.ttl", format="turtle")

for stmt in g:
    pprint.pprint(stmt)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
query = """
SELECT ?iri
WHERE {
  ?iri rdf:type <http://data.biographynet.nl/rdf/BioDes>.
}
"""

# Execute the SPARQL query
Bio_Des_list = g.query(query)

# Process the results
for row in Bio_Des_list:
    iri = row["iri"]
    print(iri)

http://data.biographynet.nl/rdf/BioDes-10000083_01
http://data.biographynet.nl/rdf/BioDes-10000083_02
http://data.biographynet.nl/rdf/BioDes-1000108_01
http://data.biographynet.nl/rdf/BioDes-1000108_02
http://data.biographynet.nl/rdf/BioDes-10004074_01
http://data.biographynet.nl/rdf/BioDes-10004074_02
http://data.biographynet.nl/rdf/BioDes-10004074_03
http://data.biographynet.nl/rdf/BioDes-10004074_04
http://data.biographynet.nl/rdf/BioDes-10004074_05
http://data.biographynet.nl/rdf/BioDes-10004074_06
http://data.biographynet.nl/rdf/BioDes-10004074_07
http://data.biographynet.nl/rdf/BioDes-1000758_01
http://data.biographynet.nl/rdf/BioDes-1000758_02
http://data.biographynet.nl/rdf/BioDes-10007696_01
http://data.biographynet.nl/rdf/BioDes-10007696_02
http://data.biographynet.nl/rdf/BioDes-10007696_03
http://data.biographynet.nl/rdf/BioDes-10007920_01
http://data.biographynet.nl/rdf/BioDes-10007920_02
http://data.biographynet.nl/rdf/BioDes-10007920_03
http://data.biographynet.nl/rdf/Bio

In [29]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT *
WHERE {
  ?iri rdf:type ?object.
  FILTER (CONTAINS(STR(?object), "Person")).
  FILTER(!CONTAINS(STR(?iri), "PersonDes"))
  FILTER(!CONTAINS(STR(?iri), "bnode"))
} 
"""

# Execute the SPARQL query
Persons_list = g.query(query)

# Process the results
for row in Persons_list:
    iri = row["iri"]
    print(iri)

http://data.biographynet.nl/rdf/Person-10000083
http://data.biographynet.nl/rdf/Person-1000108
http://data.biographynet.nl/rdf/Person-10004074
http://data.biographynet.nl/rdf/Person-1000758
http://data.biographynet.nl/rdf/Person-10007696
http://data.biographynet.nl/rdf/Person-10007920
http://data.biographynet.nl/rdf/Person-10008380
http://data.biographynet.nl/rdf/Person-10008408
http://data.biographynet.nl/rdf/Person-10010273
http://data.biographynet.nl/rdf/Person-10011885
http://data.biographynet.nl/rdf/Person-10011930
http://data.biographynet.nl/rdf/Person-10012154
http://data.biographynet.nl/rdf/Person-10012658
http://data.biographynet.nl/rdf/Person-10013970
http://data.biographynet.nl/rdf/Person-10015608
http://data.biographynet.nl/rdf/Person-10016201
http://data.biographynet.nl/rdf/Person-10017363
http://data.biographynet.nl/rdf/Person-10017386
http://data.biographynet.nl/rdf/Person-10017530
http://data.biographynet.nl/rdf/Person-10018783
http://data.biographynet.nl/rdf/Person-100

In [30]:
def get_person_numbers(person_iri):
    first_split = person_iri.split('/')
    person = first_split[4]
    second_split = person.split('-')
    person_numbers = second_split[1]
    return person_numbers

In [31]:
def get_bio_des_numbers(bio_des_iri):
    first_split = bio_des_iri.split('/')
    bio_des = first_split[4]
    second_split = bio_des.split('-')
    numbers = second_split[1]
    third_split = numbers.split('_')
    bio_des_numbers = third_split[0]
    return bio_des_numbers

In [32]:
def make_person_list(person_iri, person_numbers):
    person_list = []
    person_list.append(person_iri)
    for row in Bio_Des_list:
        iri_bio_des = row["iri"]
        str_iri_bio_des = str(iri_bio_des)
        bio_des_numbers = get_bio_des_numbers(str_iri_bio_des)
        if person_numbers == bio_des_numbers:
            person_list.append(iri_bio_des)
    return person_list

In [33]:
person_and_biodes_list = []

for row in Persons_list:
    iri_person = row["iri"]
    str_iri_person = str(iri_person)
    person_numbers = get_person_numbers(str_iri_person)
    person_list = make_person_list(iri_person, person_numbers)
    person_and_biodes_list.append(person_list)

In [34]:
def create_named_graph(description, proxy_predicate, real_person):
    graph = Graph()
    # create a graph for a description
    # add all triples with subject that matches the description
    graph += g.triples((description, None, None))
    for stmt in graph:
        subject = stmt[2]
        graph += g.triples((subject, None, None))
    graph.add((description, proxy_predicate, real_person))
    return graph

In [35]:
def create_filename(person):
    firstsplit = str(person).split('/')
    str_person_number = str(firstsplit[4])
    file_name = 'D:\\' + str_person_number + '_named_graph.ttl'
    return file_name

In [36]:
predicate = URIRef('http://data.biographynet.nl/rdf/aggregatedPerson')

for person_list in person_and_biodes_list:
    person = person_list[0]
    file_name = create_filename(person)
    person_list.pop(0)
    for bio_des in person_list:
        new_graph = create_named_graph(URIRef(bio_des), predicate, URIRef(person))
    new_graph.serialize(destination=file_name, format = 'turtle')